In [7]:
from copy import deepcopy, copy
import os
import pathlib
from random import shuffle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from livelossplot import PlotLosses

from chord_recognition.augmentations import SemitoneShift
from chord_recognition.dataset import ChromaDataset, BatchIterator, flatten_iterator, prepare_datasource, split_datasource
from chord_recognition.cnn import model

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 5)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
datasource = prepare_datasource(('queen',))
train_size = int(0.8 * len(datasource))
test_size = len(datasource) - train_size
train_ds, test_ds = split_datasource(datasource, [train_size, test_size])
print('training length=',len(train_ds), 'testing length=', len(test_ds))

shift = SemitoneShift(p=1.0, max_shift=4, bins_per_semitone=2)
train_dataset = ChromaDataset(train_ds, window_size=8192, hop_length=4096, transform=shift)

test_dataset = ChromaDataset(test_ds, window_size=8192, hop_length=4096)

loader_train = DataLoader(train_dataset, shuffle=True, num_workers=0, batch_size=64)
loader_val = DataLoader(test_dataset, num_workers=0, batch_size=64)

dataloaders = {
    "train": loader_train,
    "val": loader_val
}

training length= 16 testing length= 4


In [3]:
def save_checkpoint(state_dict, is_best, filename='new_etd_best_model.pt'):
    if is_best:
        torch.save(state_dict, f'chord_recognition/models/{filename}')

In [4]:
def train_model(model, optimizer, dataloaders, device, epochs=1):
    liveloss = PlotLosses()
    model = model.to(device=device)
    
    best_acc = 0
    for e in range(epochs):
        logs = {}
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # put model to training mode
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
        
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device=device, dtype=torch.float32)
                labels = labels.to(device=device, dtype=torch.long)
                labels = torch.argmax(labels, 1)
                if phase == 'train':
                    # Zero out all of the gradients for the variables which the optimizer
                    # will update.
                    optimizer.zero_grad()
            
                scores = model(inputs)
                scores = scores.squeeze(3).squeeze(2)                
                loss = F.cross_entropy(scores, labels)
                
                _, preds = torch.max(scores, 1)
                running_corrects += torch.sum(preds == labels)

        
                if phase == 'train':        
                    # This is the backwards pass: compute the gradient of the loss with
                    # respect to each  parameter of the model.
                    loss.backward()
        
                    # Actually update the parameters of the model using the gradients
                    # computed by the backwards pass.
                    optimizer.step()
            
                running_loss += loss.detach() * inputs.size(0)
        
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)
            prefix = ''
            if phase == 'val':
                prefix = 'val_'
                is_best = epoch_acc > best_acc
                best_acc = max(epoch_acc, best_acc)
                save_checkpoint(model.state_dict(), is_best)
            
            logs[prefix + ' log loss'] = epoch_loss.item()
            logs[prefix + 'accuracy'] = epoch_acc.item()
        
        liveloss.update(logs)
        liveloss.send()

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=1e-7)

train_model(model, optimizer, dataloaders, device, 50)

KeyboardInterrupt: 

In [ ]:
# Eliminate overfitting:
# - Emulates a slight detuning by shifting the spectrogram by fractions of up to 0.4 of a semitone
# - Try smaller sets of features
# - Try increasing lambda